In [227]:
import mistletoe
import yaml
import re
from mistletoe.base_renderer import BaseRenderer
from mistletoe.latex_renderer import LaTeXRenderer
from random import shuffle
from crypt import vigenere_encrypt

In [232]:
from mistletoe import block_token, span_token, Document
from itertools import chain

class Error(Exception):
    """Base class for exceptions in this module."""
    pass

# Define a custom renderer back to markdown so that it can be further processed 
# into latex later
class QuestionMarker(span_token.SpanToken):
    pattern = re.compile(r"\[([ |x])\]\s+(.*)")    

    def __init__(self, match):
        self.marker = match.group(1)
        self.question = match.group(2)
        
class QuestionList(block_token.List):
    pattern = re.compile(r'(?:\d{0,9}[.)]|[+\-*]) {0,1}\[[ |x]\](?:[ \t]*$|[ \t]+)')

class QuestionRenderer(LaTeXRenderer):   
    def __init__(self, *extras, **kwargs):
        """
        Args:
            extras (list): allows subclasses to add even more custom tokens.
        KeywordArgs:
            language: document language (to be used with polyglossia)
            studentno: matriculation number of the student
            studentname: name of the student
            exam: exam name
            date: exam date
        """
        self.record_answers = False
        self.answers = []
        # TODO: check parameter coherence
        self.parameters = kwargs
        super().__init__(*chain([QuestionMarker, QuestionList], extras))
        
    def render_question_marker(self, token):
        assert self.record_answers, "Probably a misplaced question marker has been used (i.e., a list not starting with it)"
        if token.marker != ' ':
            self.answers[-1].append(True)
        else:
            self.answers[-1].append(False)            
        return token.question
        
    def render_table_row(self, token):
        cells = [self.render(child) for child in token.children]
        return ' & '.join(cells) + ' \\\\\n'
    
    def render_heading(self, token):
        if token.level > 2:
            return super().render_heading(self, token)
        if token.level == 1:
            return ''
        template = "\question\n{inner}"
        return template.format(inner=self.render_inner(token))
    
    def render_question_list(self, token):
        template = " \\omrchoices{{{choiceno}}}\n\\begin{{choices}}\n{inner}\n\\end{{choices}}\n"
        self.record_answers = True
        self.answers.append([])
        answers = ['\n\\choice {inner}'.format(inner=self.render_list_item(child)) for child in token.children]        
        self.record_answers = False
        inner = ''.join(answers)
        return template.format(inner=inner, choiceno=len(self.answers[-1]))        
    
    def render_list_item(self, token):
        if not self.record_answers:
            return super().render_list_item(token)
        else:
            return "".join(self.render(child) for child in token.children)
        #    raise Error("Once a question list is started all the list items must be questions")                       
        
    @staticmethod
    def render_thematic_break(token):
        return ''

    def render_document(self, token):
        template = '''
\\documentclass{{omrexam}}
\\usepackage{{polyglossia}}
\\setdefaultlanguage{{{language}}}

\\examname{{{exam}}}
\\student{{{studentno}}}{{{studentname}}}
\\date{{{date}}}
\\solution{{{solution}}}

\\begin{{document}}
\\begin{{questions}}
{inner}
\\end{{questions}}
\\end{{document}}
'''
        self.footnotes.update(token.footnotes)
        inner = self.render_inner(token)    
        solutions = []
        for q in self.answers:
            solution = ""
            for i in range(len(q)):
                if q[i]:
                    solution += chr(ord('A') + i)
            solutions.append(solution)
        return template.format(inner=inner, 
                               solution=vigenere_encrypt(','.join(solutions), key=self.parameters['studentno']), 
                               **self.parameters)

In [233]:
filename = 'html.md'
with open('questions/{}'.format(filename), 'r') as f: 
    with QuestionRenderer(language="italian", date="20/07/2019", 
                          exam="Fondamenti di Informatica", studentno="34601",
                          studentname="Luca Di Gaspero") as renderer:
    #questions = re.compile(r"\n---").split(f.read())
        print(renderer.render(Document(f)))


\documentclass{omrexam}
\usepackage{polyglossia}
\setdefaultlanguage{italian}

\examname{Fondamenti di Informatica}
\student{34601}{Luca Di Gaspero}
\date{20/07/2019}
\solution{edhcaedhc}

\begin{document}
\begin{questions}
\question
Cos'è un attributo? \omrchoices{3}
\begin{choices}

\choice 
un parametro usato all'interno dei tag per specificare informazioni aggiuntive

\choice 
una proprietà di un'entità

\choice 
il colore di sfondo di una pagina $\phi = \sqrt{2}$

\end{choices}
\question
Cos'è un'ancora? \omrchoices{3}
\begin{choices}

\choice 
un tag HTML che specifica un collegamento

\choice 
il testo associato al riferimento evidenziato nel documento

\choice 
una ripetizione di uno o più tag

\end{choices}
\question
Di cosa si occupa il World-Wide Web Consortium (W3C) \omrchoices{3}
\begin{choices}

\choice 
di definire standard mondiali riguardanti il Web

\choice 
l'assegnazione dei nomi di dominio

\choice 
gestisce gli indirizzi IP statici

\end{choices}
\question
Un coo